# Set up OpenAI and start with a system prompt

In [62]:
import json
import os

from openai import OpenAI

In [63]:
EXAMPLES_DIR = 'examples'
os.makedirs(EXAMPLES_DIR, exist_ok=True)

In [64]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=open('../key.txt', 'r').read().strip('\n'),
)

In [65]:
system_prompt = """
# Instructions:
You are an autocomplete system for Python. Generate or complete Python code based on the user's description or the provided code snippet.

The user may
1. Describe code, either directly or in a Python comment. In this case, you should implement the described code.
2. Write incomplete code. In this case, you should complete the code.

## Example 1:

[INPUT]
# Function to add two numbers
[/INPUT]

[ANSWER]
```python
def add_numbers(a, b):
    return a + b
```
[/ANSWER]

## Example 2:

[INPUT]
I need a function that checks if a number is prime
[/INPUT]

[ANSWER]
```python
def is_prime(number):
    if number > 1:
        for i in range(2, int(number**0.5)+1):
            if (number % i) == 0:
                return False
        return True
    else:
        return False
```
[/ANSWER]

## Example 3:

[INPUT]
```python
def factorial(n):
    # Computes the fact
```
[/INPUT]

[ANSWER]
```python
    # Computes the factorial of a number
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
```
[/ANSWER]

# Rules:
- Your entire response must be valid Python code.
- Any comments or notes must be made in Python code block, i.e., ```python ... ```.
- You must not add any additional comments or notes outside of the ```python ... ``` blocks. Your entire response must only consist of one Python code block.
- You must end your response after ending the Python code block with "```".
"""

In [66]:
def add_message(messages: list[str], role: str, message: str | None = None) -> list[str]:
    # If the user is the assistant, generate a response
    if role == "assistant" and message is None:
        chat_completion = client.chat.completions.create(messages=messages, model="gpt-3.5-turbo")
        message = chat_completion.choices[0].message.content

    # Add the message to the messages list
    messages.append({
        "role": role,
        "content": message,
    })

    return messages

# Generate a response to an example prompt

In [67]:
user_input = """import json
import os

from openai import OpenAI

EXAMPLES_DIR = 'examples'
os.makedirs(EXAMPLES_DIR, exist_ok=True)

# Load your API key from an environment variable or secret management servic
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=open('../key.txt', 'r').read().strip('\n'),
)

def add_message(messages: list[str], role: str, message: str | None = None) -> list[str]:
    # If the user is the assistant, g"""

In [68]:
messages = []

In [69]:
messages = add_message(messages, "system", system_prompt)
messages = add_message(messages, "user", user_input)

In [70]:
messages = add_message(messages, "assistant")

# Examine the output and store the conversation and a comment

In [71]:
print(messages[-1]['content'])

```python
def add_message(messages: list[str], role: str, message: str | None = None) -> list[str]:
    # If the user is the assistant, just append the message to the messages list
    if role == "assistant":
        messages.append(message)
    return messages
```



In [72]:
comment = """
+ Valid python code
- Does not understand that a new message should be generated by the model if the user is the assistant
- ignores 
"""

title = "OpenAI add_message in-line autocomplete with few-shot system prompt"

In [73]:
# Remove any previous comments
messages = [message for message in messages if message['role'] != 'comment']

# Add the comment to the messages list
messages.append({
    "role": "comment",
    "content": comment,
})

In [74]:
# Save the messages to a file
# with open(os.path.join(EXAMPLES_DIR, f"{title}.json"), "w") as f:
#     json.dump(messages, f, indent=4)